# 位置显著性分析主程序

本notebook用于分析RNAscope数据中基因表达的位置分布显著性，包括：
1. 数据加载和预处理
2. 位置分布区间创建
3. 蒙特卡洛模拟分析
4. 多重比较校正
5. 结果可视化和保存

**注意**: 本程序专门处理位置分布显著性分析

In [9]:
# 导入必要的库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from modules.judge_sig import (
    create_position_bins,
    two_step_stratified_sample,
    monte_carlo_simulation,
    x_monte,
    correct_grid_pvalues,
    plot_corrected_p_heatmap,
)

print('✅ 模块导入完成！')

✅ 模块导入完成！


In [10]:
# 分析参数
n_simulations = 10000
num_xbins = 20
num_ybins = 10
correction_method = 'fdr'
bx_min = -0.6
bx_max = 0.6
by_min = -0.6
by_max = 0.6

cell_num = 0
random_num =42

sample_type = 'wt'
# ----------------------
# 设置路径和参数
# ----------------------
base_folder = 'your_pathway/vg_space_mapping/wt/' 
summary_folder = os.path.join(base_folder, "summary")
sample_info_path = os.path.join(summary_folder, "sample_info.csv")
wt_summary_path = os.path.join(summary_folder, sample_type + "_summary_all.csv")
results_file_path = os.path.join(summary_folder, 'vg_averageing_labels.csv')
vg_size_path = os.path.join(summary_folder, 'vg_size.csv')

# 打印配置信息
print(f'根目录: {base_folder}')
print(f'汇总文件夹: {summary_folder}')
print(f'模拟次数: {n_simulations}')
print(f'x分箱数量: {num_xbins}, y分箱数量 ：{num_ybins}')
print(f'校正方法: {correction_method}')

# ----------------------
# 数据加载和预处理
# ----------------------
def load_data():
    """加载所有必要数据"""
    try:
        wt_data = pd.read_csv(wt_summary_path)
        print(f"成功加载{sample_type}数据，共{len(wt_data)}行")
        
        results_data = pd.read_csv(results_file_path, header=None)
        BX = results_data.iloc[:, 0].values
        BY = results_data.iloc[:, 1].values
        print(f"成功加载BX/BY数据，长度分别为: {len(BX)}, {len(BY)}")
        
        return wt_data, BX, BY
    except Exception as e:
        print(f"数据加载错误: {e}")
        raise

# 加载基础数据
summary_data, BX, BY = load_data()
# 读取并处理WT汇总数据

original_count = len(summary_data)
print(f"去重前记录数: {original_count}")
# 去除所有列信息完全重复的行（一行所有信息都相同时才去重）
summary_data = summary_data.drop_duplicates(keep='first')
print("使用所有列进行完全重复行的去重（仅当一行所有信息均相同时才移除）")
deduplicated_count = len(summary_data)
removed_count = original_count - deduplicated_count
print(f"去重后记录数: {deduplicated_count} (移除了{removed_count}条完全重复的记录)")

# 读取vg大小数据并计算标准化坐标
vg_size = pd.read_csv(vg_size_path, header=None)
avg_vg_size = vg_size.mean(axis=0).values
print(f"vg大小平均值: {avg_vg_size}")

# 定义is_positive_cols - 找到所有以_isPositive结尾的列
is_positive_cols = [col for col in summary_data.columns if col.endswith('_isPositive')]
if not is_positive_cols:
    raise ValueError("未找到任何以'_isPositive'结尾的列，请检查数据列名")
print(f"找到{len(is_positive_cols)}个阳性标记列: {is_positive_cols}")

# 查找坐标列
x_col = next((col for col in summary_data.columns if col.endswith('warpedROIvar1')), None)
y_col = next((col for col in summary_data.columns if col.endswith('warpedROIvar2')), None)

if not x_col or not y_col:
    missing = []
    if not x_col: missing.append("warpedROIvar1结尾的列")
    if not y_col: missing.append("warpedROIvar2结尾的列")
    raise ValueError(f"缺少必要的坐标列: {', '.join(missing)}")

print(f"找到坐标列: X={x_col}, Y={y_col}")

# 计算标准化坐标
summary_data['standardized_x'] = summary_data[x_col] / avg_vg_size[0]
summary_data['standardized_y'] = summary_data[y_col] / avg_vg_size[1]

data = summary_data[['Label'] + is_positive_cols + ['standardized_x', 'standardized_y', 'hasPositive']].copy()

if cell_num > 0 :
    data = two_step_stratified_sample(data, cell_num, random_num)

order_list=[0, 7, 2, 6, 4, 5, 3, 1, 0]
BX_ordered = BX[order_list]
BY_ordered = BY[order_list]
cs = CubicSpline(np.arange(len(BX_ordered)), np.c_[BX_ordered, BY_ordered], 
                axis=0, bc_type='periodic')
t_fine = np.linspace(0, len(BX_ordered)-1, 300)
x_fine, y_fine = cs(t_fine).T

# 计算BX和BY的边界
actual_bx_min, actual_bx_max = x_fine.min(), x_fine.max()
actual_by_min, actual_by_max = y_fine.min(), y_fine.max()

# 创建区间
bx_bins, by_bins = create_position_bins(
    actual_bx_min, actual_bx_max,
    actual_by_min, actual_by_max,
    num_xbins, num_ybins
)

print(f"BX区间: {actual_bx_min:.2f} 到 {actual_bx_max:.2f}")
print(f"BY区间: {actual_by_min:.2f} 到 {actual_by_max:.2f}")


根目录: your_pathway/vg_space_mapping/wt/
汇总文件夹: your_pathway/vg_space_mapping/wt/summary
模拟次数: 10000
x分箱数量: 20, y分箱数量 ：10
校正方法: fdr
成功加载wt数据，共3604行
成功加载BX/BY数据，长度分别为: 8, 8
去重前记录数: 3604
使用所有列进行完全重复行的去重（仅当一行所有信息均相同时才移除）
去重后记录数: 3604 (移除了0条完全重复的记录)
vg大小平均值: [3088.18315202 1404.93479673]
找到5个阳性标记列: ['e_isPositive', 'b_isPositive', 'd_isPositive', 'c_isPositive', 'a_isPositive']
找到坐标列: X=e_warpedROIvar1, Y=e_warpedROIvar2
x_bins：覆盖(-0.6, 0.6)，共26个分点，基础间距=0.050525
y_bins：覆盖(-0.6, 0.6)，共14个分点，基础间距=0.103153
BX区间: -0.57 到 0.44
BY区间: -0.48 到 0.55


In [11]:
print("\n===== 开始蒙特卡洛模拟 =====")
results = []

for col in is_positive_cols:
    gene = col.replace('_isPositive', '')
    x_col = 'standardized_x'
    y_col = 'standardized_y'
    
    if x_col not in data.columns or y_col not in data.columns:
        print(f"⚠️ 警告: 未找到标准化坐标列，跳过 {col}")
        continue
    
    print(f"\n===== 分析 {col} =====")
    print(f"使用的坐标列: X={x_col}, Y={y_col}")
    
    result = monte_carlo_simulation(data, col, x_col, y_col, bx_bins, by_bins, n_simulations, save_base_dir=os.path.join(summary_folder, "test_monte"))
    
    if result is not None:
        total_real = np.sum(result['real_counts'])
        print(f"{col} 真实计数总和: {total_real}")
        
        if total_real > 0:
            result['corrected_p'] = correct_grid_pvalues(result['p_values'], method=correction_method)
            results.append(result)
            print(f"✅ {col} 分析完成")
        else:
            print(f"❌ {col} 真实计数总和为0，不添加到结果中")
    else:
        print(f"❌ {col} 分析失败，未生成结果")


===== 开始蒙特卡洛模拟 =====

===== 分析 e_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


模拟 e_isPositive: 100%|██████████| 10000/10000 [00:08<00:00, 1160.90it/s]


e_isPositive 真实计数总和: 569.0
✅ e_isPositive 分析完成

===== 分析 b_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


模拟 b_isPositive: 100%|██████████| 10000/10000 [00:08<00:00, 1136.50it/s]


b_isPositive 真实计数总和: 509.0
✅ b_isPositive 分析完成

===== 分析 d_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


...

d_isPositive 真实计数总和: 784.0
✅ d_isPositive 分析完成

===== 分析 c_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


模拟 c_isPositive: 100%|██████████| 10000/10000 [00:08<00:00, 1122.71it/s]


c_isPositive 真实计数总和: 653.0
✅ c_isPositive 分析完成

===== 分析 a_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


模拟 a_isPositive: 100%|██████████| 10000/10000 [00:08<00:00, 1124.31it/s]

a_isPositive 真实计数总和: 1089.0
✅ a_isPositive 分析完成


In [12]:
min_count = 0
max_count = 0.02
if results:
    print("\n===== 保存分析结果 =====")
    if cell_num > 0 :
        results_folder = os.path.join(summary_folder, f"grid_pvalues_{sample_type}_{cell_num}")
        visualization_folder = os.path.join(summary_folder, f"grid_visualizations_{sample_type}_{cell_num}")
    else:
        results_folder = os.path.join(summary_folder, f"grid_pvalues_{sample_type}")
        visualization_folder = os.path.join(summary_folder, f"grid_visualizations_{sample_type}")
    os.makedirs(results_folder, exist_ok=True)
    
    for result in results:
        gene = result['gene']
        np.savetxt(os.path.join(results_folder, f'{gene}_real_counts.csv'), result['real_counts'], delimiter=',', fmt='%d')
        np.savetxt(os.path.join(results_folder, f'{gene}_corrected_p.csv'), result['corrected_p'], delimiter=',', fmt='%.6f')
        plot_corrected_p_heatmap(result, data, gene, bx_min, bx_max, by_min, by_max, BX, BY, visualization_folder, min_count, max_count)
else:
    print("\n⚠️ 没有生成任何有效结果")

print("\n===== 分析完成 =====")


===== 保存分析结果 =====
已保存计数热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\e_count.png
已保存p值热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\e_p.png
已保存计数热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\b_count.png
已保存p值热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\b_p.png
已保存计数热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\d_count.png
已保存p值热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\d_p.png
已保存计数热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\c_count.png
已保存p值热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\c_p.png
已保存计数热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\a_count.png
已保存p值热图至: your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt\a_p.png

===== 分析完成 =====


In [13]:
print("\n===== 开始x轴分布蒙特卡洛模拟 =====")
results = []

for col in is_positive_cols:
    gene = col.replace('_isPositive', '')
    x_col = 'standardized_x'
    y_col = 'standardized_y'
    
    if x_col not in data.columns or y_col not in data.columns:
        print(f"⚠️ 警告: 未找到标准化坐标列，跳过 {col}")
        continue
    
    print(f"\n===== 分析 {col} =====")
    print(f"使用的坐标列: X={x_col}, Y={y_col}")
    
    result = x_monte(
        data, col, x_col, 
        bx_bins, n_simulations
    )
    
    if result is not None:
        total_real = np.sum(result['real_counts'])
        print(f"{col} 真实计数总和: {total_real}")
        
        if total_real > 0:
            result['corrected_p'] = correct_grid_pvalues(result['p_values'], method=correction_method)
            results.append(result)
            print(f"✅ {col} 分析完成")
        else:
            print(f"❌ {col} 真实计数总和为0，不添加到结果中")
    else:
        print(f"❌ {col} 分析失败，未生成结果")


===== 开始x轴分布蒙特卡洛模拟 =====

===== 分析 e_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


X轴模拟 e_isPositive: 100%|██████████| 10000/10000 [00:17<00:00, 557.35it/s]


e_isPositive 真实计数总和: 569
✅ e_isPositive 分析完成

===== 分析 b_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


X轴模拟 b_isPositive: 100%|██████████| 10000/10000 [00:17<00:00, 565.45it/s]


b_isPositive 真实计数总和: 509
✅ b_isPositive 分析完成

===== 分析 d_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


...

d_isPositive 真实计数总和: 784
✅ d_isPositive 分析完成

===== 分析 c_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


X轴模拟 c_isPositive: 100%|██████████| 10000/10000 [00:17<00:00, 565.58it/s]


c_isPositive 真实计数总和: 653
✅ c_isPositive 分析完成

===== 分析 a_isPositive =====
使用的坐标列: X=standardized_x, Y=standardized_y


X轴模拟 a_isPositive: 100%|██████████| 10000/10000 [00:17<00:00, 574.76it/s]

a_isPositive 真实计数总和: 1089
✅ a_isPositive 分析完成


In [14]:
min_count = 0
max_count = 0.1
p_value_threshold = 0.05  # 显著性阈值
# -------------------------- 关键：定义目标Marker顺序 --------------------------
target_marker_order = ['d', 'e', 'b', 'a', 'c']  # 从上到下的顺序

if results:
    print("\n===== 1. 整合并保存所有Marker的X轴数据 =====")
    # 1.1 创建带_xbins的结果文件夹（延续之前的路径规则）
    if cell_num > 0 :
        results_folder = os.path.join(summary_folder, f"grid_pvalues_{sample_type}_{cell_num}_xbins")
        visualization_folder = os.path.join(summary_folder, f"grid_visualizations_{sample_type}_{cell_num}_xbins")
    else:
        results_folder = os.path.join(summary_folder, f"grid_pvalues_{sample_type}_xbins")
        visualization_folder = os.path.join(summary_folder, f"grid_visualizations_{sample_type}_xbins")
    os.makedirs(results_folder, exist_ok=True)
    os.makedirs(visualization_folder, exist_ok=True)

    # 1.2 提取x轴分bin边界（所有marker共用同一套x bins，取第一个有效结果的x bins逻辑）
    x_bin_edges = np.linspace(BX.min(), BX.max(), num_xbins+1)  # 直接使用原x轴分bin边界
    x_bin_labels = [f"Bin{i+1}\n[{x_bin_edges[i]:.2f}-{x_bin_edges[i+1]:.2f}]" for i in range(len(x_bin_edges)-1)]
    x_bin_num = len(x_bin_labels)  # x轴总bin数

    # 1.3 整合所有marker的real_counts和corrected_p
    # 初始化整合DataFrame（行=marker，列=x bin）
    marker_list = []
    real_counts_all = []
    corrected_p_all = []

    # ---------- 1. 计算 20 个 bin 索引 ----------
    min_bin_idx  = np.digitize(x_fine.min(), bx_bins) - 1
    max_bin_idx  = np.digitize(x_fine.max(), bx_bins) - 1
    zero_bin_idx  = np.digitize(0, bx_bins) - 1
    x_bins_indices = np.arange(min_bin_idx, min_bin_idx + 20)   # 固定 20 个
    x_offsets = x_bins_indices - zero_bin_idx                 # 相对 0 的偏移量

    # ---------- 2. 以 5 为步长、以 0 为中心，只在现有 20 个偏移量里挑 ----------
    step = 5
    low  = int(np.floor(x_offsets.min() / step) * step)
    high = int(np.ceil(x_offsets.max() / step) * step)
    tick_labels = np.arange(low, high + 1, step)              # 5 的倍数
    tick_labels = tick_labels[np.isin(tick_labels, x_offsets)]  # 必存在

    # ---------- 3. 对应绘图位置 ----------
    tick_positions = [np.where(x_offsets == lab)[0][0] for lab in tick_labels]

    for result in results:
        marker = result['gene']
        rc = result['real_counts']  # 1D数组（x bin数）
        cp = result['corrected_p']  # 1D数组（x bin数）
        
        rc = rc[min_bin_idx:max_bin_idx]
        cp = cp[min_bin_idx:max_bin_idx]
        marker_list.append(marker)
        real_counts_all.append(rc)
        corrected_p_all.append(cp)

    # 1.4 保存整合数据（CSV格式，便于后续读取和分析）
    # 保存真实计数（marker×x bin）
    rc_df = pd.DataFrame(real_counts_all, index=marker_list, columns=[f"X_Bin{i+1}" for i in range(num_xbins)])
    rc_df.index.name = "Marker"
    rc_df.to_csv(os.path.join(results_folder, "all_markers_real_counts.csv"), index=True)
    print(f"✅ 已保存所有marker真实计数：all_markers_real_counts.csv")

    # 保存校正后p值（marker×x bin）
    cp_df = pd.DataFrame(corrected_p_all, index=marker_list, columns=[f"X_Bin{i+1}" for i in range(num_xbins)])
    cp_df.index.name = "Marker"
    cp_df.to_csv(os.path.join(results_folder, "all_markers_corrected_p.csv"), index=True)
    print(f"✅ 已保存所有marker校正后p值：all_markers_corrected_p.csv")

    # 保存x轴分bin信息（便于后续绘图/解读）
    x_bin_info = pd.DataFrame({
        "Bin_Index": range(1, num_xbins+1),
        "Bin_Left": BX.min(),
        "Bin_Right": BX.max()
    })
    x_bin_info.to_csv(os.path.join(results_folder, "x_axis_bin_info.csv"), index=False)
    print(f"✅ 已保存x轴分bin信息：x_axis_bin_info.csv")


===== 1. 整合并保存所有Marker的X轴数据 =====
✅ 已保存所有marker真实计数：all_markers_real_counts.csv
✅ 已保存所有marker校正后p值：all_markers_corrected_p.csv
✅ 已保存x轴分bin信息：x_axis_bin_info.csv


In [15]:
min_count = 0
max_count = 0.1
p_value_threshold = 0.05  # 显著性阈值
# -------------------------- 关键：定义目标Marker顺序 --------------------------
target_marker_order = ['d', 'e', 'b', 'a', 'c']  # 从上到下的顺序
if cell_num > 0 :
    results_folder = os.path.join(summary_folder, f"grid_pvalues_{sample_type}_{cell_num}_xbins")
    visualization_folder = os.path.join(summary_folder, f"grid_visualizations_{sample_type}_{cell_num}_xbins")
else:
    results_folder = os.path.join(summary_folder, f"grid_pvalues_{sample_type}_xbins")
    visualization_folder = os.path.join(summary_folder, f"grid_visualizations_{sample_type}_xbins")
    
if results:
    rc_df = pd.read_csv(os.path.join(results_folder, "all_markers_real_counts.csv"), header=0, index_col = 0)
    cp_df = pd.read_csv(os.path.join(results_folder, "all_markers_corrected_p.csv"), header=0, index_col = 0)
    x_bin_info = pd.read_csv(os.path.join(results_folder, "x_axis_bin_info.csv"), header=0, index_col = 0)
    # -------------------------- 2. 绘制整合热图（所有marker合并为一张） --------------------------
    print("\n===== 2. 绘制所有Marker整合热图 =====")
    # -------------------------- 关键修改1：筛选并按目标顺序重排数据 --------------------------
    # 1. 筛选：只保留目标Marker的数据（排除不在目标列表中的Marker）
    rc_df_filtered = rc_df.loc[rc_df.index.isin(target_marker_order)]  # 计数数据筛选
    cp_df_filtered = cp_df.loc[cp_df.index.isin(target_marker_order)]  # p值数据筛选
    
    # 2. 重排序：按target_marker_order的顺序调整行索引（核心步骤）
    rc_df_ordered = rc_df_filtered.reindex(target_marker_order)  # 计数数据按目标顺序排列
    cp_df_ordered = cp_df_filtered.reindex(target_marker_order)  # p值数据按目标顺序排列
    
    # 3. 检查是否有缺失（可选：避免目标Marker无数据导致空图）
    missing_markers = [m for m in target_marker_order if m not in rc_df_ordered.index]
    if missing_markers:
        print(f"⚠️ 以下Marker无数据，已跳过：{missing_markers}")
    if rc_df_ordered.empty:
        print("⚠️ 无有效Marker数据，跳过绘图")
    else:
        # 2.1 数据预处理（基于重排序后的数据）
        rc_normalized = rc_df_ordered.div(rc_df_ordered.sum(axis=1), axis=0)  # 计数归一化
        cp_clipped = cp_df_ordered.clip(upper=p_value_threshold)  # p值截断（避免颜色溢出）

        # 2.2 创建画布（2个子图：上=计数热图，下=p值热图）
        # 高度随有效Marker数量自适应（rc_df_ordered.shape[0]即有效Marker数）
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 1*rc_df_ordered.shape[0]))
        fig.suptitle(f"All Markers - X-axis Bins Analysis (Sample: {sample_type})", fontsize=14, y=0.98)

        # 2.3 绘制「计数比例热图」（上子图，基于重排序数据）
        im1 = ax1.imshow(rc_normalized.values, cmap='Blues', aspect='auto', vmin=min_count, vmax=max_count)
        # 设置坐标轴标签（yticks用重排序后的Marker列表）
        ax1.set_xticks(tick_positions)
        ax1.set_xticklabels(tick_labels, fontsize=15)
        ax1.set_xlabel("offset from border", fontsize=15)
        
        # -------------------------- 关键修改2：y轴标签用重排序后的Marker --------------------------
        ax1.set_yticks(range(len(rc_df_ordered)))  # y轴刻度位置（对应重排序后的行数）
        ax1.set_yticklabels([word.capitalize() for word in rc_df_ordered.index], fontsize=15)  # y轴标签=目标顺序的Marker
        ax1.set_title("Positive Cell Proportion", fontsize=15, pad=8)

        # 添加计数热图颜色条
        cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8, aspect=20)
        cbar1.outline.set_visible(False)
        cbar1.set_label("Cell Proportion", fontsize=15, labelpad=8)
        cbar1.set_ticks([min_count, (min_count+max_count)/2, max_count])
        cbar1.set_ticklabels([f"{x:.2f}" for x in [min_count, (min_count+max_count)/2, max_count]], fontsize=12)

        # 2.4 绘制「校正后p值热图」（下子图，基于重排序数据）
        im2 = ax2.imshow(cp_clipped.values, cmap='Reds_r', aspect='auto', vmin=0, vmax=p_value_threshold)
        # 设置坐标轴标签（y轴与上子图一致，保持顺序统一）
        ax2.set_xticks(tick_positions)
        ax2.set_xticklabels(tick_labels, fontsize=15)
        ax2.set_xlabel("offset from border", fontsize=15)
        
        # -------------------------- 关键修改3：p值图y轴与计数图保持一致顺序 --------------------------
        ax2.set_yticks(range(len(cp_df_ordered)))  # y轴刻度位置（对应重排序后的行数）
        ax2.set_yticklabels([word.capitalize() for word in cp_df_ordered.index], fontsize=15)  # y轴标签=目标顺序的Marker
        ax2.set_title(f"q-value (FDR, cutoff at {p_value_threshold})", fontsize=15, pad=8)

        # 添加p值热图颜色条
        cbar2 = plt.colorbar(im2, ax=ax2, shrink=0.8, aspect=20)
        cbar2.outline.set_visible(False)
        cbar2.set_label("FDR", fontsize=10, labelpad=8)
        cbar2.set_ticks([0, p_value_threshold/2, p_value_threshold])
        cbar2.set_ticklabels([f"{x:.2f}" for x in [0, p_value_threshold/2, p_value_threshold]], fontsize=10)

        # 2.5 调整布局（避免标签重叠）
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # 预留顶部标题空间

        # 2.6 保存整合热图
        heatmap_path = os.path.join(visualization_folder, "all_markers_xbin_combined_heatmap")
        plt.savefig(heatmap_path + ".png", dpi=1200, bbox_inches='tight')
        plt.savefig(heatmap_path + ".pdf", dpi=1200, bbox_inches='tight')
        plt.savefig(heatmap_path + ".eps", format = 'eps', dpi=1200, bbox_inches='tight')
        plt.close(fig)
        print(f"✅ 已保存所有marker整合热图：{heatmap_path}")

else:
    print("\n⚠️ 没有生成任何有效结果，跳过数据保存和绘图")

print("\n===== 分析完成 =====")


===== 2. 绘制所有Marker整合热图 =====
✅ 已保存所有marker整合热图：your_pathway/vg_space_mapping/wt/summary\grid_visualizations_wt_xbins\all_markers_xbin_combined_heatmap

===== 分析完成 =====


In [16]:
custom_color = {'d': "#E3776B",'e': "#9AA218",'b': "#1BAF73",'a': "#26A2D5",'c': "#B374AD"}

## 🎯 位置显著性分析完成！

所有分析步骤已完成，包括：
- ✅ 数据加载和预处理
- ✅ 位置分布区间创建
- ✅ 蒙特卡洛模拟分析
- ✅ 多重比较校正
- ✅ 结果保存和可视化

**生成的文件**:
- 位置显著性结果CSV文件
- 热图可视化图片

**如果遇到问题**:
1. 检查模块导入是否正确
2. 验证数据文件路径
3. 查看错误信息进行调试

**下一步**: 可以查看生成的结果文件和图片，或进行其他数据分析。